In [1]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import pandas as pd
import numpy as np

In [3]:
wiki = load_dataset('wiki_snippets', 'wiki40b_en_100_0', split='train')
eli5 = load_dataset('eli5', split='train_eli5')

In [5]:
random_question_id = np.random.randint(0, len(eli5))
random_question_id

35649

In [6]:
import psycopg2

conn = psycopg2.connect(database="eli5",
                        host="127.0.0.1",
                        user="admin",
                        password="1234",
                        port="6000")

cursor = conn.cursor()

query = 'SELECT * FROM wiki40b;'

cursor.execute(query)
data =cursor.fetchall()

In [7]:
sbert_model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [8]:
def cosine_similarity(x, y):
    x = np.asarray(x, dtype='float64')
    y = np.asarray(y, dtype='float64')
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [9]:
def sort_key(item):
    return item[2]

In [10]:
question = eli5[random_question_id]['title']
q_embed = sbert_model.encode(question)

In [11]:
for item_idx in range(len(data)):
    ref_emb = data[item_idx][1]
    ref = ref_emb[1:-1].split(',')
    ref = np.array([float(x) for x in ref])
    assert type(ref) == type(q_embed)
    item_similarity = cosine_similarity(q_embed, ref)
    data[item_idx] = data[item_idx] + (item_similarity,)
data.sort(key = lambda row: row[2], reverse=True)

In [12]:
top_5 = data[:5]

In [13]:
df = pd.DataFrame({
    question: [wiki[x]['passage_text'] for x in [x[0] for x in top_5]] 
}
)

In [14]:
df.style.set_properties(**{'text-align': 'left'})

,The theory of parallel worlds/universes
0,"relativity, which was according to him an accumulation of ""absurdities"" (like the constancy of the speed of light) and ""mathematical fictions""."
1,the universe besides Earth.
2,be confused with descriptions of the terrestrial hemispheres of Earth itself.
3,"of Goodman's paradox and were also used for the analysis of the thought experiments and paradoxes related to the Doomsday argument. In the typology of n-universes, it is worth distinguishing: - according to whether they comprise constant-criteria or/and variable-criteria (space, time, color, shape, temperature, etc.) - according to whether they comprise one or more objects - according to whether a given criterion is or not with demultiplication - according to whether the objects are in relation one-one or many-one with a given criterion The n-universes proceed of a double inspiration: on the one hand, as a system of criteria, that of Nelson Goodman and on"
4,the display of ambiguous and mysterious imagery.
